<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing\_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


## Objectives

Web scrap Falcon 9 launch records with `BeautifulSoup`:

*   Extract a Falcon 9 launch records HTML table from Wikipedia
*   Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [102]:
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)
response.status_code

200

In [105]:
response2 = requests.get('https://api.spacexdata.com/v4/launches/past')

response2.json()

[{'fairings': {'reused': False,
   'recovery_attempt': False,
   'recovered': False,
   'ships': []},
  'links': {'patch': {'small': 'https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png',
    'large': 'https://images2.imgbox.com/40/e3/GypSkayF_o.png'},
   'reddit': {'campaign': None,
    'launch': None,
    'media': None,
    'recovery': None},
   'flickr': {'small': [], 'original': []},
   'presskit': None,
   'webcast': 'https://www.youtube.com/watch?v=0a_00nJ_Y88',
   'youtube_id': '0a_00nJ_Y88',
   'article': 'https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html',
   'wikipedia': 'https://en.wikipedia.org/wiki/DemoSat'},
  'static_fire_date_utc': '2006-03-17T00:00:00.000Z',
  'static_fire_date_unix': 1142553600,
  'net': False,
  'window': 0,
  'rocket': '5e9d0d95eda69955f709d1eb',
  'success': False,
  'failures': [{'time': 33,
    'altitude': None,
    'reason': 'merlin engine failure'}],
  'details': 'Engine failure at 33 seconds and loss of vehicle',
  'crew

In [107]:
data=pd.json_normalize(response2.json())
data

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2021-08-26T02:49:00.000Z,1.629946e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],SpaceX's 23rd ISS resupply mission on behalf o...,[],[5ea6ed2d080df4000697c904],...,None,None,[],[https://live.staticflickr.com/65535/514114359...,None,https://youtu.be/x-KiDqxAMU0,x-KiDqxAMU0,None,https://en.wikipedia.org/wiki/SpaceX_CRS-23,NaN
133,2021-09-02T17:29:00.000Z,1.630604e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],None,[],[5ea6ed30080df4000697c913],...,None,https://www.reddit.com/r/spacex/comments/k2ts1...,[],[https://live.staticflickr.com/65535/514748536...,None,https://youtu.be/4372QYiPZB4,4372QYiPZB4,https://spaceflightnow.com/2021/09/14/spacex-l...,https://en.wikipedia.org/wiki/Starlink,NaN
134,2021-09-13T07:07:00.000Z,1.631517e+09,False,18000.0,5e9d0d95eda69973a809d1ec,True,[],Inspiration4 is the world’s first all-civilian...,"[607a3a5f5a906a44023e0870, 607a3ab45a906a44023...","[5ea6ed2f080df4000697c910, 5ee68c683c228f36bd5...",...,None,None,[],[],None,https://youtu.be/3pv01sSq44w,3pv01sSq44w,None,https://en.wikipedia.org/wiki/Inspiration4,NaN
135,2021-10-28T05:46:00.000Z,1.635400e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],None,"[5fe3c587b3467846b3242198, 5fe3c5beb3467846b32...","[5ea6ed2d080df4000697c904, 5ee68c683c228f36bd5...",...,None,None,[],[],None,https://youtu.be/WZvtrnFItNs,WZvtrnFItNs,None,https://en.wikipedia.org/wiki/SpaceX_Crew-3,NaN


In [109]:
data.columns

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')

Create a `BeautifulSoup` object from the HTML `response`


In [10]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup=BeautifulSoup(response.text,'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly


In [1]:
# Use soup.title attribute
#soup.prettify()

In [97]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [ ]:
, soup.find_all('p')[2].get_text()

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [16]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables=soup.find_all('table')
html_tables

[<table class="multicol" role="presentation" style="border-collapse: collapse; padding: 0; border: 0; background:transparent; width:100%;">
 <tbody><tr>
 <td style="text-align: left; vertical-align: top;">
 <h3><span class="mw-headline" id="Rocket_configurations">Rocket configurations</span></h3>
 <div class="chart noresize" style="margin-top:1em;max-width:420px;">
 <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
 <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
 <div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2"></div>
 <div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-p

Starting from the third table is our target table contains the actual launch records.


In [17]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [41]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

for col_name in first_launch_table.find_all('th',{ "scope" : "col" }):
      if len(col_name.text) is not None and len(col_name.text) > 0:
        print(col_name.text)
        column_names.append(col_name.text.strip())

Flight No.

Date andtime (UTC)

Version,Booster [b]

Launch site

Payload[c]

Payload mass

Orbit

Customer

Launchoutcome

Boosterlanding



In [42]:
column_names

['Flight No.',
 'Date andtime (UTC)',
 'Version,Booster [b]',
 'Launch site',
 'Payload[c]',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 'Boosterlanding']

Check the extracted column names


In [43]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


In [46]:
dict.fromkeys(column_names)

{'Flight No.': None,
 'Date andtime (UTC)': None,
 'Version,Booster [b]': None,
 'Launch site': None,
 'Payload[c]': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launchoutcome': None,
 'Boosterlanding': None}

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [53]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date andtime (UTC)']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [54]:
launch_dict

{'Flight No.': [],
 'Version,Booster [b]': None,
 'Launch site': [],
 'Payload[c]': None,
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launchoutcome': None,
 'Boosterlanding': None,
 'Payload': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [90]:
extracted_row = 0
launch_dict=[]
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #launch_dict['Flight No.'].append(flight_number)
            
            print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            
            date = datatimelist[0].strip(',')
            #launch_dict['Date'].append(date)
          
            print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
           # launch_dict['Time'].append(time)
            
            print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            #launch_dict['Version Booster'].append(bv)
          
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print(launch_site)
            #launch_dict['Launch site'].append(launch_site)
            
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print(payload)
            #launch_dict['Payload'].append(payload)
            
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            print(payload_mass)
            #launch_dict['Payload mass'].append(payload_mass)
          
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            print(orbit)
            #launch_dict['Orbit'].append(orbit)
            
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            print(customer)
            #launch_dict['Customer'].append(customer)
            launch_dict_row={'Flight No.':flight_number,'Date':date,'Time':time,'Version Booster':bv,'Launch site':launch_site,'Payload':payload,'Payload mass':payload_mass,'Orbit':orbit,'Customer':customer}
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0].strip()
            print(launch_outcome)
            launch_dict_row['Launch outcome']=launch_outcome
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8]).string.strip()
            print(booster_landing)
            #launch_dict.append['Booster landing'].append(booster_landing)
            launch_dict_row['Booster landing']=booster_landing
           
            launch_dict.append(launch_dict_row)
            print(launch_dict_row)
            

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success
Failure
{'Flight No.': '1', 'Date': '4 June 2010', 'Time': '18:45', 'Version Booster': 'F9 v1.0B0003.1', 'Launch site': 'CCAFS', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch outcome': 'Success', 'Booster landing': 'Failure'}
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
0
LEO
NASA
Success
Failure
{'Flight No.': '2', 'Date': '8 December 2010', 'Time': '15:43', 'Version Booster': 'F9 v1.0B0004.1', 'Launch site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Booster landing': 'Failure'}
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
525 kg
LEO
NASA
Success
No attempt
{'Flight No.': '3', 'Date': '22 May 2012', 'Time': '07:44', 'Version Booster': 'F9 v1.0B0005.1', 'Launch site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': '525 kg', 'Orbit': 'LEO',

AttributeError: 'NoneType' object has no attribute 'string'

In [91]:
launch_dict

[{'Flight No.': '1',
  'Date': '4 June 2010',
  'Time': '18:45',
  'Version Booster': 'F9 v1.0B0003.1',
  'Launch site': 'CCAFS',
  'Payload': 'Dragon Spacecraft Qualification Unit',
  'Payload mass': 0,
  'Orbit': 'LEO',
  'Customer': 'SpaceX',
  'Launch outcome': 'Success',
  'Booster landing': 'Failure'},
 {'Flight No.': '2',
  'Date': '8 December 2010',
  'Time': '15:43',
  'Version Booster': 'F9 v1.0B0004.1',
  'Launch site': 'CCAFS',
  'Payload': 'Dragon',
  'Payload mass': 0,
  'Orbit': 'LEO',
  'Customer': 'NASA',
  'Launch outcome': 'Success',
  'Booster landing': 'Failure'},
 {'Flight No.': '3',
  'Date': '22 May 2012',
  'Time': '07:44',
  'Version Booster': 'F9 v1.0B0005.1',
  'Launch site': 'CCAFS',
  'Payload': 'Dragon',
  'Payload mass': '525 kg',
  'Orbit': 'LEO',
  'Customer': 'NASA',
  'Launch outcome': 'Success',
  'Booster landing': 'No attempt'},
 {'Flight No.': '4',
  'Date': '8 October 2012',
  'Time': '00:35',
  'Version Booster': 'F9 v1.0B0006.1',
  'Launch sit

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [95]:
df=pd.DataFrame(launch_dict)
df

,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure
1,2,8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt
3,4,8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,No attempt
4,5,1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,No attempt
...,...,...,...,...,...,...,...,...,...,...,...
100,101,6 December 2020,16:17:08,F9 B5 ♺,KSC,SpaceX CRS-21,"2,972 kg",LEO,NASA,Success,Success
101,102,13 December 2020,17:30:00,F9 B5 ♺,CCSFS,SXM-7,"7,000 kg",GTO,Sirius XM,Success,Success
102,103,19 December 2020,14:00:00,F9 B5 ♺,KSC,NROL-108,C,LEO,NRO,Success,Success
103,104,8 January 2021,02:15,F9 B5,CCSFS,Türksat 5A,"3,500 kg",GTO,Türksat,Success,Success


In [96]:
df.set_index(df.columns[0])
df

,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,4 June 2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure
1,2,8 December 2010,15:43,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,22 May 2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt
3,4,8 October 2012,00:35,F9 v1.0B0006.1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,No attempt
4,5,1 March 2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,No attempt
...,...,...,...,...,...,...,...,...,...,...,...
100,101,6 December 2020,16:17:08,F9 B5 ♺,KSC,SpaceX CRS-21,"2,972 kg",LEO,NASA,Success,Success
101,102,13 December 2020,17:30:00,F9 B5 ♺,CCSFS,SXM-7,"7,000 kg",GTO,Sirius XM,Success,Success
102,103,19 December 2020,14:00:00,F9 B5 ♺,KSC,NROL-108,C,LEO,NRO,Success,Success
103,104,8 January 2021,02:15,F9 B5,CCSFS,Türksat 5A,"3,500 kg",GTO,Türksat,Success,Success


In [100]:
df['Version Booster'].value_counts()

F9 B5             25
F9 FT             17
F9 v1.1           14
F9 B4              7
F9 B5 ♺            4
F9 FTB1035.2       1
F9 B4B1045.2       1
F9 B5B1049.6       1
F9 B5B1046.2       1
F9 B5B1046.1       1
F9 B5B1049.3       1
F9 B5B1060.2       1
F9 B5B1058.2       1
F9 B5B1048.3       1
F9 B4B1043.2       1
F9 v1.0B0004.1     1
F9 B5B1047.2       1
F9 FTB1036.2       1
F9 B4B1040.2       1
F9 FTB1038.2       1
F9 FTB1031.2       1
F9 v1.1B1003       1
F9 B5[268]         1
F9 B5B1051.2       1
F9 B4B1039.2       1
F9 FTB1032.2       1
F9 B5B1051.6       1
F9 B5B1058.3       1
F9 B5B1046.3       1
F9 B5B1048         1
F9 B4B1041.2       1
F9 v1.0B0003.1     1
F9 FTB1029.2       1
F9 B5B1056.2       1
F9 v1.0B0006.1     1
F9 B5B1049.2       1
F9 v1.0B0005.1     1
F9 B5B1047.3       1
F9 B5B1048.2       1
F9 v1.0B0007.1     1
F9 B5B1056.3       1
F9 B5B1051.8       1
F9 FT♺             1
Name: Version Booster, dtype: int64

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates               |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
